In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-02-10 14:45:40--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  6.18MB/s    in 4.8s    

2023-02-10 14:45:45 (5.21 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

inspect data and remove nan values

In [4]:
df_books.shape

(271379, 3)

In [5]:
df_ratings.shape

(1149780, 3)

In [6]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [7]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [8]:
df_books.isnull().sum()

isbn      0
title     0
author    1
dtype: int64

In [9]:
df_books.dropna(inplace=True)

In [10]:
df_books.isnull().sum()

isbn      0
title     0
author    0
dtype: int64

In [11]:
df_ratings.isnull().sum()

user      0
isbn      0
rating    0
dtype: int64

Ensure statistical significance through removing users with less than 200 ratings and books with less than 100 ratings

In [12]:
# Remove users with less than 200 ratings
grouped_users = df_ratings.groupby('user')['rating'].count()
filtered_users = grouped_users[grouped_users >= 200]
users = filtered_users.index.tolist()
df_ratings_cleaned = df_ratings[df_ratings['user'].isin(users)]

In [13]:
df_ratings_cleaned.shape

(527556, 3)

In [14]:
# Remove books with less than 100 ratings
grouped_books = df_ratings['isbn'].value_counts()
filtered_books = grouped_books[grouped_books >= 100]
books = filtered_books.index.tolist()
df_ratings_cleaned = df_ratings_cleaned[df_ratings_cleaned['isbn'].isin(books)]

In [15]:
df_ratings_cleaned.shape

(49781, 3)

In [16]:
df_ratings_cleaned.head()

,user,isbn,rating
1456,277427,002542730X,10.0
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1484,277427,0140067477,0.0


create pivot table for model

In [17]:
pivot_table = df_ratings_cleaned.pivot_table(values=['rating'], index=['isbn'], columns=['user']).fillna(0)
pivot_table.head()

rating                                                          \
user       254    2276   2766   2977   3363   4017   4385   6242   6251     
isbn                                                                        
002542730X    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
0060008032    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
0060096195    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
006016848X    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
0060173289    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

                   ...                                                   \
user       6323    ... 274004 274061 274301 274308 274808 275970 277427   
isbn               ...                                                    
002542730X    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0   10.0   
0060008032    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
0060096195    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
006016848X    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
0060173289    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

                                 
user       277478 277639 278418  
isbn                             
002542730X    0.0    0.0    0.0  
0060008032    0.0    0.0    0.0  
0060096195    0.0    0.0    0.0  
006016848X    0.0    0.0    0.0  
0060173289    0.0    0.0    0.0  

[5 rows x 888 columns]

In [18]:
model = NearestNeighbors(metric='cosine')
model.fit(pivot_table.values)

NearestNeighbors(metric='cosine')

create function to pass in book title and return 5 nearest neighbors

In [19]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  isbn = df_books.loc[df_books['title'] == book]['isbn'].values[0]
  distance, indice = model.kneighbors([pivot_table.loc[isbn].values], n_neighbors=5)
  nn = pivot_table.iloc[indice[0]].index.values
  recommended_books = []
  for i in nn:
    recommended_books.append([df_books.loc[df_books['isbn'] == i]['title'].values[0], distance[0][nn.tolist().index(i)]])
  
  def comp(item):
    return item[1]

  recommended_books = sorted(recommended_books, key=comp, reverse=True)
  recommended_books.pop()
  return [book, recommended_books]

test function

In [20]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
